# Preparing SDSS DR12 VACs

## Introduction

Working with the SDSS DR12 VAC files in preparation for loading into a database. There are two VACs currently targeted for loading into Data Lab:

1. [DR12Q](https://www.sdss.org/dr16/data_access/value-added-catalogs/?vac_id=the-sloan-digital-sky-survey-quasar-catalog:-fourteenth-data-release)
2. [Portsmouth Emission Line Catalog](https://www.sdss.org/dr16/data_access/value-added-catalogs/?vac_id=portsmouth-stellar-kinematics-and-emission-line-fluxes)

All of these are conveniently stored as single FITS tables.  However, varying degrees of preprocessing may be needed.

Other resources:

* [SDSS CAS SQL definition files](https://trac.sdss.org/browser/repo/sdss/sas/trunk/sql)


## Setup

In [4]:
# Standard library
import os
from io import BytesIO
import re
import csv
# matplotlib, etc.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.font_manager import fontManager, FontProperties
import numpy as np
from astropy.io import fits
from astropy.visualization import astropy_mpl_style
from astropy.io.votable import parse_single_table
from astropy.table import Column, Table
plt.style.use(astropy_mpl_style)
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)
# Data Lab
# from dl import queryClient as qc, storeClient as sc
#
# Global constants.
#
release = 12
sas = 'sdss_dr{0:d}://'.format(release)
sdss_run2d = 26
boss_run2d = 'v5_7_0'
# assert matplotlib.rcParams['figure.figsize'] == (10.0, 10.0)

In [30]:
def sdss_specobjid(plate, fiber, mjd, run2d, line=None, index=None):
    """Convert SDSS spectrum identifiers into CAS-style specObjID.
    Bits are assigned in specObjID thus:
    ===== ========== =============================================================
    Bits  Name       Comment
    ===== ========== =============================================================
    50-63 Plate ID   14 bits
    38-49 Fiber ID   12 bits
    24-37 MJD        Date plate was observed minus 50000 (14 bits)
    10-23 run2d      Spectroscopic reduction version
    0-9   line/index 0 for use in SpecObj files see below for other uses (10 bits)
    ===== ========== =============================================================
    Parameters
    ----------
    plate, fiber, mjd : :class:`int` or array of int
        Plate, fiber ID, and MJD for a spectrum.  If arrays are
        passed, all must have the same length.  The MJD value must be
        greater than 50000.
    run2d : :class:`int`, :class:`str` or array of int or str
        The run2d value must be an integer or a string of the form 'vN_M_P'.
        If an array is passed, it must have the same length as the other
        inputs listed above.  If the string form is used, the values are
        restricted to :math:`5 \le N \le 6`, :math:`0 \le M \le 99`,
        :math:`0 \le P \le 99`.
    line : :class:`int`, optional
        A line index, only used for defining specObjID for SpecLine files.
        `line` and `index` cannot both be non-zero.
    index : :class:`int`, optional
        An index measure, only used for defining specObjID for SpecLineIndex
        files. `line` and `index` cannot both be non-zero.
    Returns
    -------
    :class:`numpy.ndarray` of :class:`numpy.uint64`
        The specObjIDs of the objects.
    Raises
    ------
    :exc:`ValueError`
        If the sizes of the arrays don't match or if the array values are
        out of bounds.
    Notes
    -----
    * On 32-bit systems, makes sure to explicitly declare all inputs as
      64-bit integers.
    * This function defines the SDSS-III/IV version of specObjID, used for
      SDSS DR8 and subsequent data releases.  It is not compatible with
      SDSS DR7 or earlier.
    * If the string form of `run2d` is used, the bits are assigned by
      the formula :math:`(N - 5) \\times 10000 + M \\times 100 + P`.
    Examples
    --------
    >>> from pydl.pydlutils.sdss import sdss_specobjid
    >>> print(sdss_specobjid(4055,408,55359,'v5_7_0'))
    [4565636362342690816]
    """
    if line is not None and index is not None:
        raise ValueError("line and index inputs cannot both be non-zero!")
    if isinstance(plate, int):
        plate = np.array([plate], dtype=np.uint64)
    if isinstance(fiber, int):
        fiber = np.array([fiber], dtype=np.uint64)
    if isinstance(mjd, int):
        mjd = np.array([mjd], dtype=np.uint64) - 50000
    if isinstance(run2d, str):
        try:
            run2d = np.array([int(run2d)], dtype=np.uint64)
        except ValueError:
            # Try a "vN_M_P" string.
            m = re.match(r'v(\d+)_(\d+)_(\d+)', run2d)
            if m is None:
                raise ValueError("Could not extract integer run2d value!")
            else:
                N, M, P = m.groups()
            run2d = np.array([(int(N) - 5)*10000 + int(M) * 100 + int(P)],
                             dtype=np.uint64)
    elif isinstance(run2d, int):
        run2d = np.array([run2d], dtype=np.uint64)
    if line is None:
        line = np.zeros(plate.shape, dtype=plate.dtype)
    else:
        if isinstance(line, int):
            line = np.array([line], dtype=np.uint64)
    if index is None:
        index = np.zeros(plate.shape, dtype=plate.dtype)
    else:
        if isinstance(index, int):
            index = np.array([index], dtype=np.uint64)
    #
    # Check that all inputs have the same shape.
    #
    if plate.shape != fiber.shape:
        raise ValueError("fiber.shape does not match plate.shape!")
    if plate.shape != mjd.shape:
        raise ValueError("mjd.shape does not match plate.shape!")
    if plate.shape != run2d.shape:
        raise ValueError("run2d.shape does not match plate.shape!")
    if plate.shape != line.shape:
        raise ValueError("line.shape does not match plate.shape!")
    if plate.shape != index.shape:
        raise ValueError("index.shape does not match plate.shape!")
    #
    # Check ranges of parameters
    #
    if ((plate < 0) | (plate >= 2**14)).any():
        raise ValueError("plate values are out-of-bounds!")
    if ((fiber < 0) | (fiber >= 2**12)).any():
        raise ValueError("fiber values are out-of-bounds!")
    if ((mjd < 0) | (mjd >= 2**14)).any():
        raise ValueError("MJD values are out-of-bounds!")
    if ((run2d < 0) | (run2d >= 2**14)).any():
        raise ValueError("MJD values are out-of-bounds!")
    if ((line < 0) | (line >= 2**10)).any():
        raise ValueError("line values are out-of-bounds!")
    if ((index < 0) | (index >= 2**10)).any():
        raise ValueError("index values are out-of-bounds!")
    #
    # Compute the specObjID
    #
    specObjID = ((plate << 50) |
                 (fiber << 38) |
                 (mjd << 24) |
                 (run2d << 10) |
                 (line | index))
    return specObjID

## DR12Q

### Overview

* [data model](https://data.sdss.org/datamodel/files/BOSS_QSO/DR12Q/DR12Q.html)
* [Pâris et al. (2017)](https://ui.adsabs.harvard.edu/abs/2017A%26A...597A..79P/abstract)

### Preprocessing Plan

* Compute `specObjID`; add `run2d` column.
* Delete photometric information.
* Move duplicate information to separate table.
* Coerce some matching flags to integer.
* Coerce floating point values (except `RA`, `DEC`) to 32-bit float.

### Known Problems (from DR14Q)

* The columns `GALEX_MATCHED`, `UKIDSS_MATCHED` are stored as float when they should be integers.  The also contain NaN values, and `GALEX_MATCHED` contains some values which are 2.  None of these cases are described in the DR14Q paper.  Only `FIRST_MATCHED` has valid values and is consistent with its description in the paper.
* The photometric id values, `RUN_NUMBER`, `COL_NUMBER`, are all nonsense values.  All identically zero in fact.
* The duplicate column contains extraneous zeroes, and not all duplicates are actually stored in the column.  See below for examples.  It would be much better if the duplicates were a separate table, mapping primary specObjID to duplicate specObjID.
* `TUNIT` columns should be blank when the value has no unit, instead of `-`.
* I would recommend checking that all units in the table conform to the FITS standard.  For example 'Vega' is a description of a unit, not a physical unit. Also, the `RASS_COUNTS` column has units counts/s, which is misleading, because the units are actually log(counts/s).  Exception: nanomaggies are not standard, but are OK.
* Column names, *e.g.* `FLUX_0.2_2.0keV` contain characters that could be interpreted as integers.
* Not all "primary" objects in DR14Q correspond to real objects in the specObjAll table.  And not all "duplicates" correspond to real objects either.  In these cases there are spectra that exist on disk, but are not now, and probably never will be added to CAS.  This makes it particularly difficult to cleanly join DR14Q to specObjAll.

In [9]:
# qso_version = 'v4_4'
# vopath = "{0}eboss/qso/DR{1:d}Q/DR{1:d}Q_{2}.fits".format(sas, release, qso_version)
# with BytesIO(sc.get(vopath, mode='binary')) as q:
    # with fits.open(q) as hdulist:
        # dr14q = hdulist['DR14Q_v4_4'].data
q = os.path.join(os.environ['HOME'], 'Documents', 'Data', 'sdss', 'dr{0:d}'.format(release), 'boss', 'qso', 'DR{0:d}Q'.format(release), 'DR{0:d}Q.fits'.format(release))
with fits.open(q) as hdulist:
    dr12q = hdulist['DR12Q.fits'].data

In [10]:
dr12q['PLATE'].min(), dr12q['PLATE'].max()

(3586, 7517)

In [11]:
dr12q['SDSS_NAME'][0]

'000000.45+174625.4'

## Need plate information

Although all DR12Q spectra appear to be from BOSS, there are two possible BOSS `run2d` values in DR12.  And some plates appear in both!

In [12]:
p = os.path.join(os.environ['HOME'], 'Documents', 'Data', 'sdss', 'dr{0:d}'.format(release), 'sdss', 'spectro', 'redux', 'plates-dr{0:d}.fits'.format(release))
with fits.open(p) as hdulist:
    platex = hdulist[1].data

In [13]:
platex

FITS_rec([('299489402097264640', 'dr7', '26', '', '122',  266,   122,   -1, -1, 51602, '51602', 145.89281  ,  6.13090000e-02, 'NOCVS:iop:/usrdevel/scott/iop', 'SPEC2 v4_5', '06:08:33.65', '2000-02-28', 'tai', -0.8279895 ,  0.56074258,  1.07004371e-03,  4, 4.45843669e+09, 4.45843392e+09, 4.45843847e+09, 1.20134, 5159901, '0266-51599-01', 'plPlugMapM-0266-51599-01.par', 3600.  , 3600.  , 3600.  , 3600.  , 3600.  , 'v5_3_12', 'v5_3_12', 'v5_3_12', 'sdss', 'chunk4', 'SDSS', 'legacy', 'chunk4', 'sdss', 'SDSS-I, -II; ctile=v1_0; v2_2', 'bad', 'standard Legacy criteria',  7.67533, -9999.    , 16, 4, 4, 4, 4, 20.858 , 16.1727, 13.4718, 13.3075, 10.6615,  7.67533, -9999.    , -9999.    , -9999.    , -9999.    , -9999.    , -9999.    , -9999., -1,  4.973931 , -0.0481682 , 0.0568238, -0.0242577 , 0.0559449, -0.0175524, 0.0639471, -0.00223064, 0.0281125, -0.00537586, 0.0187705, 0.00730515, 0.0539542, -0.00434113, 0.0482151, 0.00234509, 0.0540306, 0.0101013 , 0.0356505, -0.00299549, 0.0293739, 26, 

## specObjID

In [28]:
run2d = np.array([boss_run2d.encode('utf-8')]*len(dr12q))
run2d_lookup = dict()
for k, row in enumerate(dr12q):
    pmjd = "{0}-{1}".format(row['PLATE'], row['MJD'])
    if pmjd in run2d_lookup:
        run2d[k] = run2d_lookup[pmjd]
        continue
    # q = "SELECT run2d FROM sdss_dr{0:d}_new.platex WHERE plate = {1:d};".format(release, p)
    # vot = qc.query(sql=q, fmt='votable', async=False)
    # result = parse_single_table(BytesIO(vot.encode('utf-8')))
    # print(result.array['run2d'][0])
    r = np.unique(platex[(platex['PLATE'] == row['PLATE']) & (platex['MJD'] == row['MJD'])]['RUN2D'])
    # print(pmjd, r)
    run2d[k] = run2d_lookup[pmjd] = r[0]


In [51]:
irun2d = np.zeros(dr12q.shape, dtype=np.uint64)
for r in ('v5_7_0', 'v5_7_2'):
    w = run2d == r.encode('utf-8')
    assert w.sum() > 0
    m = re.match(r'v(\d+)_(\d+)_(\d+)', r)
    N, M, P = m.groups()
    irun2d[w] = (int(N) - 5)*10000 + int(M) * 100 + int(P)
specobjid = sdss_specobjid(dr12q['PLATE'].astype(np.uint64),
                           dr12q['FIBERID'].astype(np.uint64),
                           dr12q['MJD'].astype(np.uint64) - 50000,
                           irun2d)
sdss_joinid = sdss_specobjid(dr12q['PLATE'].astype(np.uint64),
                             dr12q['FIBERID'].astype(np.uint64),
                             dr12q['MJD'].astype(np.uint64) - 50000,
                             np.zeros(irun2d.shape, dtype=np.uint64))


In [59]:
(dr12q['NSPEC_BOSS'] == 3).sum()

1089

## Handle duplicates

In [61]:
with open(os.path.join(os.environ['HOME'], 'Downloads', 'dr12q_duplicates.csv'), 'w', newline='') as csvfile:
    dw = csv.writer(csvfile)
    dw.writerow(['specobjid', 'dupspecobjid', 'run2d', 'plate', 'mjd', 'fiberid', 'snr'])
    for k in range(len(dr12q)):
        if dr12q[k]['NSPEC_BOSS'] > 0:
            w = dr12q['PLATE_DUPLICATE'][k, :] > 0
            Nd = w.sum()
            irun2d = np.zeros((Nd,), dtype=np.uint64)
            plates = dr12q['PLATE_DUPLICATE'][k, w]
            mjds = dr12q['MJD_DUPLICATE'][k, w]
            fibers = dr12q['FIBERID_DUPLICATE'][k, w]
            snrs = dr12q['SNR_DUPLICATE'][k, w]
            pmjds = ['{0:d}-{1:d}'.format(plates[j], mjds[j]) for j in range(Nd)]
            for i, p in enumerate(plates):
                try:
                    foo = run2d_lookup[pmjds[i]]
                except KeyError:
                    # q = "SELECT run2d FROM sdss_dr{0:d}_new.platex WHERE plate = {1:d} AND mjd = {2:d};".format(release, p, mjds[i])
                    # try:
                    #     vot = qc.query(sql=q, fmt='votable', async=False)
                    # except qc.queryClientError as e:
                    #     if str(e) == 'Error: SQL query did not return any records':
                    #         q = "SELECT run2d FROM sdss_dr{0:d}_new.platex WHERE plate = {1:d};".format(release, p)
                    #         vot = qc.query(sql=q, fmt='votable', async=False)
                    #     else:
                    #         raise
                    # result = parse_single_table(BytesIO(vot.encode('utf-8')))
                    # foo = run2d_lookup[p] = result.array['run2d'][0]
                    foo = run2d_lookup[pmjds[i]] = np.unique(platex[(platex['PLATE'] == plates[i]) & (platex['MJD'] == mjds[i])]['RUN2D'])[0]
                # if foo == boss_run2d.encode('utf-8'):
                #     irun2d[i] = br
                # else:
                #     irun2d[i] = int(foo)
                m = re.match(r'v(\d+)_(\d+)_(\d+)', foo)
                N, M, P = m.groups()
                br = (int(N) - 5)*10000 + int(M) * 100 + int(P)
                irun2d[i] = br
            dupspecobjid = sdss_specobjid(plates.astype(np.uint64),
                                          fibers.astype(np.uint64),
                                          mjds.astype(np.uint64) - 50000,
                                          irun2d)
            for l in range(Nd):
                dw.writerow([specobjid[k].astype(np.int64),
                             dupspecobjid[l].astype(np.int64),
                             run2d_lookup[pmjds[l]],
                             plates[l],
                             mjds[l],
                             fibers[l],
                             snrs[l]])

## Preprocess main table

In [65]:
t = Table(dr12q)
for c in dr12q.columns.names:
    if dr12q[c].dtype.type is np.float64 and c != 'RA' and c != 'DEC':
        t[c] = dr12q[c].astype(np.float32)
# t['GALEX_MATCHED'] = galex_matched
# t['UKIDSS_MATCHED'] = ukidss_matched
t.remove_columns(['PLATE_DUPLICATE', 'MJD_DUPLICATE', 'FIBERID_DUPLICATE', 'SNR_DUPLICATE',
                  'PHOTO_MJD', 'RUN_NUMBER', 'RERUN_NUMBER', 'COL_NUMBER', 'FIELD_NUMBER', 'OBJ_ID'])
t.add_column(Column(name='SPECOBJID', data=specobjid.astype(np.character)), index=0)
t.add_column(Column(name='SDSS_JOINID', data=sdss_joinid.astype(np.character)), index=1)
t.add_column(Column(name='RUN2D', data=run2d), index=6)


In [66]:
t

SPECOBJID,SDSS_JOINID,SDSS_NAME,RA,DEC,THING_ID,RUN2D,PLATE,MJD,FIBERID,Z_VI,Z_PIPE,ERR_ZPIPE,ZWARNING,Z_PCA,ERR_ZPCA,PCA_QUAL,Z_CIV,Z_CIII,Z_MGII,SDSS_MORPHO,BOSS_TARGET1,ANCILLARY_TARGET1,ANCILLARY_TARGET2,EBOSS_TARGET0,NSPEC_BOSS,SDSS_DR7,PLATE_DR7,MJD_DR7,FIBERID_DR7,UNIFORM,ALPHA_NU,SNR_SPEC,SNR_1700,SNR_3000,SNR_5150,FWHM_CIV,BHWHM_CIV,RHWHM_CIV,AMP_CIV,REWE_CIV,ERR_REWE_CIV,FWHM_CIII,BHWHM_CIII,RHWHM_CIII,AMP_CIII,REWE_CIII,ERR_REWE_CIII,FWHM_MGII,BHWHM_MGII,RHWHM_MGII,AMP_MGII,REWE_MGII,ERR_REWE_MGII,BAL_FLAG_VI,BI_CIV,ERR_BI_CIV,AI_CIV,ERR_AI_CIV,CHI2TROUGH,NCIV_2000,VMIN_CIV_2000,VMAX_CIV_2000,NCIV_450,VMIN_CIV_450,VMAX_CIV_450,REW_SIIV,REW_CIV,REW_ALIII,PSFFLUX [5],IVAR_PSFFLUX [5],PSFMAG [5],ERR_PSFMAG [5],TARGET_FLUX [5],MI,DGMI,EXTINCTION [5],EXTINCTION_RECAL [5],HI_GAL,VAR_MATCHED,VAR_CHI2,VAR_A,VAR_GAMMA,RASS_COUNTS,RASS_COUNTS_SNR,SDSS2ROSAT_SEP,N_DETECTION_XMM,FLUX02_12KEV_SGL,ERR_FLUX02_12KEV_SGL,FLUX02_2KEV,ERR_FLUX02_2KEV,FLUX2_12KEV,ERR_FLUX2_12KEV,FLUX02_12KEV,ERR_FLUX02_12KEV,LUM02_2KEV_SGL,LUM05_2KEV,LUM2_12KEV,LUM02_2KEV,LUMX2_10_UPPER,SDSS2XMM_SEP,GALEX_MATCHED,FUV,FUV_IVAR,NUV,NUV_IVAR,JMAG,ERR_JMAG,JSNR,JRDFLAG,HMAG,ERR_HMAG,HSNR,HRDFLAG,KMAG,ERR_KMAG,KSNR,KRDFLAG,SDSS2MASS_SEP,W1MAG,ERR_W1MAG,W1SNR,W1CHI2,W2MAG,ERR_W2MAG,W2SNR,W2CHI2,W3MAG,ERR_W3MAG,W3SNR,W3CHI2,W4MAG,ERR_W4MAG,W4SNR,W4CHI2,CC_FLAGS,PH_FLAG,SDSS2WISE_SEP,UKIDSS_MATCHED,YFLUX,YFLUX_ERR,JFLUX,JFLUX_ERR,HFLUX,HFLUX_ERR,KFLUX,KFLUX_ERR,FIRST_MATCHED,FIRST_FLUX,FIRST_SNR,SDSS2FIRST_SEP
bytes20,bytes20,str18,float64,float64,int32,bytes6,int32,int32,int32,float32,float32,float32,int32,float32,float32,float32,float32,float32,float32,int16,int64,int64,int64,int64,int32,int32,int32,int32,int32,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,float32,int32,float32,float32,int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,int16,float32,float32,float32,float32,float32,float32,float32,int32,float32,float32,float32,int32,float32,float32,float32,int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,str4,str4,float32,int16,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32
6950325365131374592,6950325365130657792,000000.45+174625.4,0.001898285183756343,17.773739129913793,268514930,v5_7_0,6173,56238,528,2.3090973,2.3090973,0.000943339,0,2.3076386,-1.0,0.3461823,2.3123078,2.306385,2.3046865,0,2199023796224,0,0,0,0,0,-1,-1,-1,0,0.53733623,0.7794849,0.5856829,0.9155019,-1.0,4278.79,1595.2887,2683.5015,2.1532874,44.002865,1.4009717,3575.001,1554.1217,2020.8794,0.70666695,38.428566,15.851724,7996.404,4579.19,3417.2139,0.22327954,69.386,17.880812,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,1.038477 .. 2.6584318,12.359663 .. 0.8528282,22.439793 .. 21.362684,0.28713453 .. 0.38640517,1.038477 .. 2.6584318,-23.88293,-0.08209104,0.14619489 .. 0.041944176,0.113614365 .. 0.03385113,21.929712,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,1,0.30231887,8.785573,0.7275649,4.5373588,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,0.0,0.0,0.0
6954847382081826816,6954847382081110016,000000.66+145828.8,0.0027564300929725505,14.974675493723005,245101352,v5_7_0,6177,56268,595,2.4979408,2.4979408,0.0008752802,0,2.5158746,0.0015261462,0.06378377,2.4965057,2.489434,-1.0,0,2199023255552,0,0,0,1,0,-1,-1,-1,0,-1.0268776,3.5009146,2.8351843,1.0002095,-1.0,5336.114,2014.5121,3321.6018,3

In [67]:
t.write(os.path.join(os.environ['HOME'], 'Downloads', 'dr12q_datalab.fits'))